In [34]:
#importing pandas library and readinginsurance data set
import pandas as pd
df =pd.read_csv('insurance_pre.csv')

In [35]:
df
#from this data set age , sex,ami , children,smoker and all input charges is output

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [36]:
#smoker and sex is categorical column so using get_dummies function performaing one hot encoding to change it to number
df = pd.get_dummies(df,drop_first=True)
df

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [37]:
df.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [38]:
# splitting input and output column
x_independent = df[['age', 'bmi', 'children',  'sex_male', 'smoker_yes']]
y_depenedent = df[['charges']]

In [39]:
#splitting train and test set
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_independent,y_depenedent,test_size=0.30,random_state=0)

In [40]:
#converting into standard form
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_s = sc.fit_transform(x_train) 
x_test_s = sc.transform(x_test)

In [57]:
# create function
#adaboosting checking 
def AdaBoost_checking(x_train,x_test,y_train,y_test,x_train_s,x_test_s):
    ''' input should be x_train , x_test,y_train y_test in standard form and acutal form
    this function will create Adaboosting Regressor model using all possible hyper parameter in normal form and standard form
     and make a note of all combo r2 core
     return value is 2 data frame 1 st data frame will give all reading 
     2nd data frame will give which combo is working good that combo details     
      
    '''
    from sklearn.ensemble import AdaBoostRegressor
    from sklearn.metrics import r2_score
    AdaBoost_score_df=pd.DataFrame(columns=['std/normal','n_estimators','r_score'])
    n_estimators = [50,100,150,200,300]
    form=['Standard','Normal']
    for f in form:
        for i in n_estimators:
            AdaBoost_reg=AdaBoostRegressor(n_estimators=i,random_state =0)
            if(f=='Standard'):
                AdaBoost_reg.fit(x_train_s,y_train)
                y_pred = AdaBoost_reg.predict(x_test_s)
            else:
                AdaBoost_reg.fit(x_train,y_train)
                y_pred = AdaBoost_reg.predict(x_test)                
            AdaBoost_r_score = r2_score(y_test,y_pred)
            AdaBoost_score_df.loc[len(AdaBoost_score_df.index)] = [f, i , AdaBoost_r_score]
            max_score= AdaBoost_score_df[AdaBoost_score_df['r_score']==AdaBoost_score_df['r_score'].max()]
    return (AdaBoost_score_df,max_score)   
            

In [58]:
#checking Adaboosting model
AdaBoost_all,AdaBoost_max=AdaBoost_checking(x_train,x_test,y_train,y_test,x_train_s,x_test_s)

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samp

In [59]:
AdaBoost_max

,std/normal,n_estimators,r_score
0,Standard,50,0.861808
1,Standard,100,0.861808
2,Standard,150,0.861808
3,Standard,200,0.861808
4,Standard,300,0.861808


In [60]:
AdaBoost_all

,std/normal,n_estimators,r_score
0,Standard,50,0.861808
1,Standard,100,0.861808
2,Standard,150,0.861808
3,Standard,200,0.861808
4,Standard,300,0.861808
5,Normal,50,0.844748
6,Normal,100,0.844748
7,Normal,150,0.844748
8,Normal,200,0.844748
9,Normal,300,0.844748


In [61]:
# this will train adaboosting model with given x and y train field with given n estimators
def AdaBoost_Model(x_train,y_train,n_estimators):
    from sklearn.ensemble import AdaBoostRegressor
    AdaBoost_reg=AdaBoostRegressor(n_estimators=n_estimators,random_state =0)            
    AdaBoost_reg.fit(x_train_s,y_train)                         
    return(AdaBoost_reg)
            

In [52]:
AdaBoost_reg=AdaBoost_Model(x_train_s,y_train,100)

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [53]:
#this model is not good for this dataset , saving adaboost regressor model for study purpose otherwise it is not helpful for acurate prediction
import pickle 
pickle.dump(AdaBoost_reg,open("insurance_premium_Adaboost.sav",'wb'))

In [54]:
#loading adaboost model
loaded_model= pickle.load(open("insurance_premium_Adaboost.sav",'rb'))

In [63]:
#as we have used standard form to train  the model input should be standardised ,so converting input into standard form
std_input = sc.transform([[19,27.900,0,0,1]])
#funcion to transform input to standard form


C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [64]:
result = loaded_model.predict(std_input)

In [65]:
result

array([18971.33402691])